Example notebook to run gPhoton to create GALEX light curves etc.

In [1]:
%matplotlib widget
import gPhoton
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as uu
from astropy.io import ascii
from astropy.table import Table

Do aperture photometry for each visit with gPhoton

In [2]:
%%time

# **** Setup analysis parameters *****

#Sources from ELAISN1_MOS15 in TDS, but not in VASCA
srcs = """recno	ID	ra	dec
71	ELAISN1_MOS15-20	242.8292	53.7492
184	ELAISN1_MOS15-24	243.0512	54.2599
307	ELAISN1_MOS15-03	242.1960	54.0721
390	ELAISN1_MOS15-22	242.8843	53.5921
417	ELAISN1_MOS15-18	242.7775	53.9643
441	ELAISN1_MOS15-13	242.4726	53.6579
495	ELAISN1_MOS15-00	241.6487	54.3255
873	ELAISN1_MOS15-14	242.6429	53.4919
934	ELAISN1_MOS15-12	242.4613	53.4860"""
tt_srcs = ascii.read(srcs)

src_idx = 0
outdir = "./TDS-ApJ_766_60/"
gapp_outfile = outdir+str(tt_srcs[src_idx]["ID"])+'_gaperture.npy'
ron = (10*uu.arcsec).to(uu.deg).value        # Radius of signal annulus
roff1 = (10*uu.arcsec).to(uu.deg).value      # Inner radius of background ring
roff2 = (20*uu.arcsec).to(uu.deg).value      # Outer radius of background ring

pos_ra = tt_srcs[src_idx]["ra"]
pos_dec = tt_srcs[src_idx]["dec"]

# **** Run analysis *****

#If running for the first time in this configuration set to True
run_first_time = False
if run_first_time:
    print(f"Running query with gFind for:\n",tt_srcs[src_idx])
    dd_gfind = gPhoton.gFind(band='NUV',skypos=[pos_ra,pos_dec]) #,maxgap=100.,minexp=100.
    t_bins = list(zip(dd_gfind["NUV"]["t0"],dd_gfind["NUV"]["t1"]))

    print("Running lightcurve with gAperture..")
    dd_gaperture = gPhoton.gAperture(band='NUV',skypos=[pos_ra,pos_dec],radius=ron, annulus=[roff1,roff2],tranges=t_bins)
    np.save(gapp_outfile, dd_gaperture)
    print("..done")
else:
    dd_gaperture = np.load(gapp_outfile,allow_pickle='TRUE').item()

Running query with gFind for:
 recno        ID           ra      dec  
----- ---------------- -------- -------
   71 ELAISN1_MOS15-20 242.8292 53.7492
NUV: 47629.0s (raw) in 44 exposures.
    [ 768413194.995, 768413297.995 ], 103.000 seconds
    [ 773123709.995, 773124202.995 ], 493.000 seconds
    [ 775242112.995, 775243663.995 ], 1551.000 seconds
    [ 799225954.995, 799227043.995 ], 1089.000 seconds
    [ 799291027.995, 799292207.995 ], 1180.000 seconds
    [ 799308775.995, 799309669.995 ], 894.000 seconds
    [ 799415397.995, 799415513.995 ], 116.000 seconds
    [ 802267370.995, 802268489.995 ], 1119.000 seconds
    [ 802374181.995, 802374993.995 ], 812.000 seconds
    [ 806071957.995, 806073409.995 ], 1452.000 seconds
    [ 806108028.995, 806108899.995 ], 871.000 seconds
    [ 806421101.995, 806422424.995 ], 1323.000 seconds
    [ 806527645.995, 806528914.995 ], 1269.000 seconds
    [ 806634281.995, 806635413.995 ], 1132.000 seconds
    [ 806693305.995, 806694584.995 ], 1279.000 s

Draw light curves

In [35]:
print(dd_gaperture.keys())
print(dd_gaperture["flags"])
keep_keys = ('t_mean', 'exptime','flux_bgsub', 'flux_bgsub_err','mag_bgsub', 'mag_bgsub_err_1', 'mag_bgsub_err_2',"flags")
dd_gapp = {x: dd_gaperture[x] for x in keep_keys if x in dd_gaperture}
tt_gapp = Table(data= dd_gapp)
tt_gapp["s2n"]=dd_gaperture['flux_bgsub']/dd_gaperture['flux_bgsub_err']
display(tt_gapp[tt_gapp["s2n"]>3.5])

dict_keys(['params', 't0', 't1', 'exptime', 't0_data', 't1_data', 't_mean', 'q_mean', 'counts', 'flat_counts', 'responses', 'detxs', 'detys', 'detrad', 'racent', 'deccent', 'mcat_bg', 'bg_counts', 'bg_flat_counts', 'bg', 'cps', 'cps_err', 'cps_bgsub', 'cps_bgsub_err', 'cps_mcatbgsub', 'cps_mcatbgsub_err', 'flux', 'flux_err', 'flux_bgsub', 'flux_bgsub_err', 'flux_mcatbgsub', 'flux_mcatbgsub_err', 'mag', 'mag_err_1', 'mag_err_2', 'mag_bgsub', 'mag_bgsub_err_1', 'mag_bgsub_err_2', 'mag_mcatbgsub', 'mag_mcatbgsub_err_1', 'mag_mcatbgsub_err_2', 'photons', 'flags'])
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.  65.   0.   0.   0.   0.   0.   4.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. 512.  32.   0.   0.   0.   0.   0.   0.
   0.   0.]


t_mean,exptime,flux_bgsub,flux_bgsub_err,mag_bgsub,mag_bgsub_err_1,mag_bgsub_err_2,flags,s2n
float64,float64,float64,float64,float64,float64,float64,float64,float64
802267923.1495593,1014.5231948290783,1.7804307836427198e-17,5.0657652348447764e-18,22.73835531435877,0.27185614395139623,0.3635133769849723,0.0,3.5146334287188403
802374589.1704702,734.4640726768934,2.0586436123474383e-17,5.6918150807960464e-18,22.58071512826742,0.26503822712795966,0.3513792773372728,0.0,3.6168490773588524
929622856.1948605,1328.7775590872222,1.6944597116512677e-17,4.000550669890856e-18,22.792089932706926,0.23013045437616242,0.29240296163319357,0.0,4.235566179436733
987629636.2642901,1276.2881343580063,4.8966987203231154e-17,5.423520237386851e-18,21.63990959102351,0.11404931664914031,0.12745094750555808,0.0,9.028635472894322
989054810.7792666,1166.4554074674954,1.6718535217083914e-17,4.4307296598126346e-18,22.806672490026127,0.2552426301473467,0.33430973477554815,0.0,3.773314217006619


In [ ]:
fig = plt.figure(figsize=(8, 6), constrained_layout=True)
ax = plt.gca()
#ax.invert_yaxis()
    
plt.errorbar(dd_gaperture['t_mean'],dd_gaperture['flux_bgsub'],yerr=dd_gaperture['flux_bgsub_err'],fmt="bo")